In [5]:
import os
import random
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageEnhance


In [ ]:
PROCESSED_DIR = "../dataset_processed"  # folder: train/ val/ test/ per kelas

print(PROCESSED_DIR)

### Augmentasi (Optional) 
augmentasi hanya untuk kelas minoritas.

In [ ]:
# Hitung jumlah gambar per kelas di train/
train_dir = os.path.join(PROCESSED_DIR, "train")
class_counts = {cls: len(os.listdir(os.path.join(train_dir, cls)))
                for cls in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, cls))}

max_count = max(class_counts.values())
print("Jumlah gambar sebelum augmentasi:", class_counts)
print("Target setiap kelas:", max_count, "\n")

# Daftar jenis augmentasi (manual + ImageDataGenerator)
AUG_TYPES = ["flip", "bright", "dark", "zoom", "rotate"]

# Proses augmentasi hanya untuk kelas minoritas
for cls, count in class_counts.items():
    if count >= max_count:
        continue  # skip kelas mayoritas

    cls_path = os.path.join(train_dir, cls)
    images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    total_needed = max_count - count
    aug_per_image = max(1, total_needed // len(images))  # jumlah augment per gambar

    print(f"Augmenting {cls}: {count} asli, tambah {total_needed} total (~{aug_per_image} per gambar)")

    total_generated = 0
    for img_name in images:
        img_path = os.path.join(cls_path, img_name)
        base_name, ext = os.path.splitext(img_name)
        img_pil = Image.open(img_path).convert("RGB")
        img_np = np.expand_dims(np.array(img_pil), 0)

        # Tentukan jenis augmentasi acak
        for i in range(aug_per_image):
            aug_type = random.choice(AUG_TYPES)
            aug_img = img_pil.copy()

            if aug_type == "flip":
                aug_img = aug_img.transpose(Image.FLIP_LEFT_RIGHT)
            elif aug_type == "bright":
                enhancer = ImageEnhance.Brightness(aug_img)
                aug_img = enhancer.enhance(1.4)
            elif aug_type == "dark":
                enhancer = ImageEnhance.Brightness(aug_img)
                aug_img = enhancer.enhance(0.7)
            elif aug_type == "zoom":
                zoom_factor = random.uniform(1.1, 1.3)
                w, h = aug_img.size
                new_w, new_h = int(w / zoom_factor), int(h / zoom_factor)
                left = (w - new_w) // 2
                top = (h - new_h) // 2
                aug_img = aug_img.crop((left, top, left + new_w, top + new_h)).resize((w, h))
            elif aug_type == "rotate":
                aug_img = aug_img.rotate(random.randint(-25, 25))

            # Simpan hasil dengan nama baru
            new_name = f"{base_name}_aug_{aug_type}_{i+1:02d}{ext.lower()}"
            aug_img.save(os.path.join(cls_path, new_name))
            total_generated += 1

            if total_generated >= total_needed:
                break
        if total_generated >= total_needed:
            break

print("\nAugmentasi merata selesai ✅\n")

# Cek ulang jumlah gambar per kelas setelah augmentasi
final_counts = {cls: len(os.listdir(os.path.join(train_dir, cls)))
                for cls in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, cls))}
print("Jumlah gambar setelah augmentasi:", final_counts)
